In [ ]:
import pandas as pd
import altair as alt
import matplotlib.pyplot as plt 
import numpy as np
import optuna
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import holidays 

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, mean_absolute_percentage_error, mean_absolute_error, mean_squared_error, r2_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

import category_encoders as ce
import xgboost as xgb
from xgboost import XGBRegressor
from ydata_profiling import ProfileReport
from sklearn.linear_model import LinearRegression

import catboost as cb
from scalecast.Forecaster import Forecaster

import lightgbm as lgb
import torch

import time

alt.data_transformers.disable_max_rows()
sklearn.set_config(transform_output="pandas")

ImportError: cannot import name 'LogitRegression' from 'sklearn.linear_model' (c:\Users\Leo\.pyenv\pyenv-win-venv\envs\venv_kaggle\lib\site-packages\sklearn\linear_model\__init__.py)

In [2]:

holidays.US(years=2025)

{datetime.date(2025, 1, 1): "New Year's Day", datetime.date(2025, 5, 26): 'Memorial Day', datetime.date(2025, 6, 19): 'Juneteenth National Independence Day', datetime.date(2025, 7, 4): 'Independence Day', datetime.date(2025, 9, 1): 'Labor Day', datetime.date(2025, 11, 11): 'Veterans Day', datetime.date(2025, 11, 27): 'Thanksgiving', datetime.date(2025, 12, 25): 'Christmas Day', datetime.date(2025, 1, 20): 'Martin Luther King Jr. Day', datetime.date(2025, 2, 17): "Washington's Birthday", datetime.date(2025, 10, 13): 'Columbus Day'}

In [3]:
train.info()

NameError: name 'train' is not defined

## Import Data

In [4]:
train = pd.read_csv('train.csv')
blind = pd.read_csv('test.csv')

In [5]:

def is_holiday(row):
    """
    Determines if a given date is a holiday in the specified country.
    Optimized for Canada, Finland, and Italy.
    
    Parameters:
    row: pandas Series containing 'country' and 'date' columns
    
    Returns:
    bool: True if the date is a holiday in the given country, False otherwise
    """
    country = row['country']
    date = pd.to_datetime(row['date']).date()  # Convert to date only

    country_mapping = {
        'Canada': holidays.CA(),
        'Finland': holidays.FI(),
        'Italy': holidays.IT(), 
        'Kenya': holidays.KE(),
        'Norway': holidays.NO(),
        'Singapore': holidays.SG(),
    }
    
    try:
        # Get the holidays object for the country
        if country in country_mapping:
            country_holidays = country_mapping[country]
            return date in country_holidays
        else:
            return False
    except Exception as e:
        print(f"Error checking holiday for {country} on {date}: {e}")
        return False

# Create a cache of holiday objects to improve performance
def initialize_holiday_detection(df):
    """
    Initialize holiday detection by creating holiday objects for all unique years in the dataset.
    This improves performance by avoiding repeated creation of holiday objects.
    
    Parameters:
    df: pandas DataFrame containing 'date' column
    """
    years = pd.to_datetime(df['date']).dt.year.unique()
    holiday_cache = {
        'Canada': {year: holidays.CA(years=year) for year in years},
        'Finland': {year: holidays.FI(years=year) for year in years},
        'Italy': {year: holidays.IT(years=year) for year in years}, 
        'Kenya': {year: holidays.KE(years=year) for year in years},
        'Norway': {year: holidays.NO(years=year) for year in years},
        'Singapore': {year: holidays.SG(years=year) for year in years}

    }
    
    def is_holiday_cached(row):
        country = row['country']
        date = pd.to_datetime(row['date']).date()
        year = date.year
        
        try:
            if country in holiday_cache:
                return date in holiday_cache[country][year]
            return False
        except Exception as e:
            print(f"Error checking holiday for {country} on {date}: {e}")
            return False
    
    return is_holiday_cached

# Example usage:
# First initialize the cached version
holiday_checker = initialize_holiday_detection(train)

# Then apply it to create the holiday flag
train['is_holiday'] = train.apply(holiday_checker, axis=1)

In [67]:
def feature_eng(df):
    df = df.copy()
    df['date'] = pd.to_datetime(df['date'])

    # Extract date features
    
    df['day_of_year'] = df['date'].dt.dayofyear
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year

    # Encode categorical features
    # Drop missing values and irrelevant columns
    imputer = SimpleImputer(strategy="mean")  # Change to "mean", "most_frequent", or "constant"
    scaler = MinMaxScaler()
    df = df.set_index('id')
    df = df.drop(columns=['date'])  # Dropping columns that are not useful for modeling
    # Separate features and target
    try: 
        df["num_sold"] = imputer.fit_transform(df[["num_sold"]])
       # df["num_sold"] = scaler.fit_transform(df[["num_sold"]])

    except: pass
    try: 
        X = df.drop(columns=['num_sold'])
        X = pd.get_dummies(X)

    except:
        X=pd.get_dummies(df)
    try:
        y = df['num_sold']
    except:
        y = None
    # Scale features
    #X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

    return df, X,  y

# Feature engineering
df, X, y = feature_eng(train)

# Splitting the data
SPLIT = 0.85
split_index = int(SPLIT * len(X))

X_train = X[:split_index]
y_train = y[:split_index]
X_test = X[split_index:]
y_test = y[split_index:]

# Verify shapes
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)


X_train shape: (195610, 18)
y_train shape: (195610,)
X_test shape: (34520, 18)
y_test shape: (34520,)


In [69]:
X_train

,is_holiday,day_of_year,month,year,country_Canada,country_Finland,country_Italy,country_Kenya,country_Norway,country_Singapore,store_Discount Stickers,store_Premium Sticker Mart,store_Stickers for Less,product_Holographic Goose,product_Kaggle,product_Kaggle Tiers,product_Kerneler,product_Kerneler Dark Mode
id,,,,,,,,,,,,,,,,,,
0,True,1,1,2010,True,False,False,False,False,False,True,False,False,True,False,False,False,False
1,True,1,1,2010,True,False,False,False,False,False,True,False,False,False,True,False,False,False
2,True,1,1,2010,True,False,False,False,False,False,True,False,False,False,False,True,False,False
3,True,1,1,2010,True,False,False,False,False,False,True,False,False,False,False,False,True,False
4,True,1,1,2010,True,False,False,False,False,False,True,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195605,False,348,12,2015,False,False,True,False,False,False,False,False,True,True,False,False,False,False
195606,False,348,12,2015,False,False,True,False,False,False,False,False,True,False,True,False,False,False
195607,False,348,12,2015,False,False,True,False,False,False,False,False,True,False,False,True,False,False


In [70]:
y_train

id
0         752.527382
1         973.000000
2         906.000000
3         423.000000
4         491.000000
             ...    
195605    150.000000
195606    874.000000
195607    717.000000
195608    453.000000
195609    438.000000
Name: num_sold, Length: 195610, dtype: float64

In [68]:
y_test


id
195610     170.0
195611    1047.0
195612     981.0
195613     513.0
195614     492.0
           ...  
230125     466.0
230126    2907.0
230127    2299.0
230128    1242.0
230129    1622.0
Name: num_sold, Length: 34520, dtype: float64

In [71]:
model = LinearRegression()

# Train the model using training data
model.fit(X_train.values, y_train.values)

# Make predictions on the test data
y_pred = model.predict(X_test.values)

# Evaluate the model using common metrics
mae = mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print evaluation metrics
print(f"Mean Absolute Error: {mae:.4f}")
print(f"MAPE: {mape:.4f}")

print(f"Mean Squared Error: {mse:.4f}")
print(f"R-squared: {r2:.4f}")

# Optionally: You can get the coefficients and intercept of the linear regression model
print(f"Coefficients: {model.coef_}")
print(f"Intercept: {model.intercept_}")

Mean Absolute Error: 248.1475
MAPE: 3.0422
Mean Squared Error: 118420.2639
R-squared: 0.6478
Coefficients: [ 3.77165272e+01  8.10233613e-02 -3.42425231e+00 -6.81727755e+00
  7.79357301e+01 -9.81025856e+00 -2.14775054e+02 -6.50285148e+02
  7.08026730e+02  8.89080004e+01 -3.08593593e+02  2.22084217e+02
  8.65093759e+01 -4.56236057e+02  4.90390358e+02  2.69673676e+02
 -1.99475825e+02 -1.04352152e+02]
Intercept: 14490.864200642352


In [74]:
def objective(trial, X_train, X_test, y_train, y_test, total_trials):
    """
    Optuna objective function for XGBoost hyperparameter optimization with GPU support.
    
    Parameters:
    trial: Optuna trial object
    X_train, X_test, y_train, y_test: Training and test data
    total_trials: Total number of trials to run
    """
    trial_start = time.time()
    print(f"🔄 Starting Trial {trial.number + 1} out of {total_trials}")
    
    # Check GPU availability
    gpu_available = torch.cuda.is_available()
    device = "gpu" if gpu_available else "cpu"
    tree_method = "hist" if gpu_available else "auto"
    
    # Define hyperparameters with reasonable ranges
    param = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 500),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2, log=True),
        "max_depth": trial.suggest_int("max_depth", 3, 18),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 0.1, 10.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 0.1, 10.0, log=True),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 7),
        "gamma": trial.suggest_float("gamma", 0, 5),
        "enable_categorical": True,
        "device": device,
        "tree_method": tree_method
    }
    
    try:
        # Convert data to device-appropriate format
        if device == "gpu":
            X_train_device = xgb.DMatrix(X_train, y_train)
            X_test_device = xgb.DMatrix(X_test)
        else:
            X_train_device = X_train
            X_test_device = X_test
        
        # Train the model with early stopping
        model = xgb.XGBRegressor(
            **param,
            early_stopping_rounds=50,
            eval_metric=['mae', 'mape']
        )
        
        model.fit(
            X_train_device, y_train,
            eval_set=[(X_test_device, y_test)],
            verbose=False
        )
        
        # Make predictions
        preds = model.predict(X_test_device)
        mape = mean_absolute_percentage_error(y_test, preds)
        
        # Calculate additional metrics
        mae = np.mean(np.abs(y_test - preds))
        rmse = np.sqrt(np.mean((y_test - preds) ** 2))
        
        # Log results
        trial.set_user_attr('mae', mae)
        trial.set_user_attr('rmse', rmse)
        trial.set_user_attr('n_estimators_used', model.best_iteration or param['n_estimators'])
        
        trial_time = time.time() - trial_start
        print(f"✅ Trial {trial.number + 1} completed in {trial_time:.1f}s")
        print(f"   MAPE: {mape:.4f}, MAE: {mae:.4f}, RMSE: {rmse:.4f}")
        
        return mape
        
    except Exception as e:
        print(f"❌ Trial {trial.number + 1} failed: {str(e)}")
        raise optuna.exceptions.TrialPruned()

def run_optimization(X_train, X_test, y_train, y_test, n_trials=5):
    """
    Run the complete optimization process and return the best model.
    """
    study = optuna.create_study(direction="minimize")
    
    try:
        study.optimize(
            lambda trial: objective(trial, X_train, X_test, y_train, y_test, n_trials),
            n_trials=n_trials,
            show_progress_bar=True
        )
        
        # Get best parameters and create final model
        best_params = study.best_params
        best_params.update({
            "enable_categorical": True,
            "device": "gpu" if torch.cuda.is_available() else "cpu",
            "tree_method": "hist" if torch.cuda.is_available() else "auto"
        })
        
        best_model = xgb.XGBRegressor(**best_params)
        best_model.fit(X_train, y_train)
        
        print("\n🏆 Best Trial Results:")
        print(f"MAPE: {study.best_value:.4f}")
        print("Best Parameters:", best_params)
        
        return best_model, study
        
    except KeyboardInterrupt:
        print("\n⚠️ Optimization interrupted by user")
        return None, study


best_model, study = run_optimization(X_train, X_test, y_train, y_test, n_trials=15)

# Make predictions with best model
if best_model is not None:
    predictions = best_model.predict(X_test)
    final_mape = mean_absolute_percentage_error(y_test, predictions)
    print(f"Final MAPE: {final_mape:.4f}")

[I 2025-01-30 13:26:17,001] A new study created in memory with name: no-name-ab083a5c-0b07-4b12-ba43-df4ce3f5a67e


  0%|          | 0/15 [00:00<?, ?it/s]

🔄 Starting Trial 1 out of 15
✅ Trial 1 completed in 2.7s
   MAPE: 1.8820, MAE: 110.6834, RMSE: 147.2241
[I 2025-01-30 13:26:19,756] Trial 0 finished with value: 1.8819735792729888 and parameters: {'n_estimators': 76, 'learning_rate': 0.021529971531891543, 'max_depth': 15, 'subsample': 0.9551041287765066, 'colsample_bytree': 0.9948239312763494, 'reg_alpha': 6.937097660895436, 'reg_lambda': 0.3382793882593333, 'min_child_weight': 2, 'gamma': 2.248265238783105}. Best is trial 0 with value: 1.8819735792729888.
🔄 Starting Trial 2 out of 15
✅ Trial 2 completed in 3.6s
   MAPE: 0.6619, MAE: 75.7065, RMSE: 120.6409
[I 2025-01-30 13:26:23,392] Trial 1 finished with value: 0.6618970142189052 and parameters: {'n_estimators': 204, 'learning_rate': 0.04510335348442594, 'max_depth': 15, 'subsample': 0.8138140076129938, 'colsample_bytree': 0.5445631689210862, 'reg_alpha': 0.6993973423962253, 'reg_lambda': 5.868284764973139, 'min_child_weight': 5, 'gamma': 1.67479363594156}. Best is trial 1 with value

In [34]:
holiday_checker = initialize_holiday_detection(blind)
blind['is_holiday'] =blind.apply(holiday_checker, axis=1)
X_trial = feature_eng(blind)[1]


In [78]:
predictions = best_model.predict(X_trial)
submission_df = pd.DataFrame({"predictions": predictions})
submission_df = pd.merge(blind['id'], 
         submission_df,
         left_index=True,
         right_index=True)
submission_file = submission_df.to_csv("submission.csv", index=False)


In [79]:
predictions

array([ 731.68994,  672.5833 ,  554.0756 , ..., 1995.6849 , 1142.4535 ,
       1370.7294 ], dtype=float32)